 ## Carga y Transformación de los archivos

 Tomaremos los archivos Excel de **homicidios.xlsx** &  **lesiones.xlsx**, haremos las transformaciones y lo dejaremos listo como CSV para su carga en SQL
 

 Como unica libreria utilizaremos Pandas, aunque será necesario tener instalado *openpyxl*

In [2]:
import pandas as pd

In [3]:
rut_hom = r'C:\Users\OS\Desktop\Proyecto Buenos aires\Data\raw\homicidios.xlsx'
rut_les = r'C:\Users\OS\Desktop\Proyecto Buenos aires\Data\raw\lesiones.xlsx'
homicidios = pd.read_excel(rut_hom)
lesiones = pd.read_excel(rut_les)

# comenzamos por Homicidios

Observamos que cuenta con varias columnas que son redundantes, y pocos valores nulos
esto de los valores nulos es confuso, hay muchos valores 'SD' (sin datos)

In [5]:
homicidios.isna().sum()


ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64

In [6]:
homicidios.head(2)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS


la columna **lugar del hecho** no cuenta con valores nulos, asi que la usaremos para crear una nueva columna **LUGAR_DEL_HECHO_NORMALIZADO** Y luego quitaremos las columnas que no interesan

In [7]:
homicidios['LUGAR_DEL_HECHO_NORMALIZADO'] = homicidios['LUGAR_DEL_HECHO'].str.upper() \
    .str.replace(' Y ', ' & ') \
    .str.replace(r'\bAV(ENIDA)?\b', '') \
    .str.replace(r'\s+', ' ') \
    .str.strip()

In [8]:
columnas = ['LUGAR_DEL_HECHO', 'Dirección Normalizada','AAAA','MM','Altura','DD','PARTICIPANTES', 'XY (CABA)', 'Cruce']
homicidios = homicidios.drop(columns=columnas)

In [9]:
columnas = homicidios.columns.tolist()
indice_calle = columnas.index('Calle')
columnas.insert(indice_calle + 1, 'LUGAR_DEL_HECHO_NORMALIZADO')
homicidios = homicidios[columnas]

Obsevamos los formatos de cada columna y nos damos cuenta lo siguiente:

**ID:**
Tiene valores numericos pero con un caracter '-' que lo quitaremos para poder volverlo numerico

**HORA:** 
Tiene falores SD que seran necesario transformar para convertirlo a formato hora en SQL

**HH:**
Contiene valores tipo SD que  impiden su paso a formato Entero

**pos x & pos y:**
representan longitud y latitud respectivamente, seran tratados y conveertidos a float para un mejor manejo

**VICTIMA,ACUSADO,CALLE:**
sus valores 'SD' seran pasados a DESCONOCIDO


In [10]:
homicidios = homicidios.loc[:, ~homicidios.columns.duplicated()]
homicidios.dtypes

ID                                     object
N_VICTIMAS                              int64
FECHA                          datetime64[ns]
HORA                                   object
HH                                     object
TIPO_DE_CALLE                          object
Calle                                  object
LUGAR_DEL_HECHO_NORMALIZADO            object
COMUNA                                  int64
pos x                                  object
pos y                                  object
VICTIMA                                object
ACUSADO                                object
dtype: object

In [11]:
homicidios['ID'] = homicidios['ID'].str.replace('-', '')
homicidios['ID'] = pd.to_numeric(homicidios['ID'])
homicidios['HH'] = pd.to_numeric(homicidios['HH'], errors='coerce')
homicidios.rename(columns={'pos x': 'longitud',
                           'pos y': 'Latitud'}, inplace= True)

In [12]:
homicidios.isnull().sum()

ID                             0
N_VICTIMAS                     0
FECHA                          0
HORA                           0
HH                             1
TIPO_DE_CALLE                  0
Calle                          1
LUGAR_DEL_HECHO_NORMALIZADO    0
COMUNA                         0
longitud                       0
Latitud                        0
VICTIMA                        0
ACUSADO                        0
dtype: int64

In [13]:
null_HH = homicidios.loc[homicidios['HH'].isnull()]
null_calle = homicidios.loc[homicidios['Calle'].isnull()]
print(null_calle)
print(null_HH)

           ID  N_VICTIMAS      FECHA      HORA    HH TIPO_DE_CALLE Calle  \
119  20160151           1 2016-11-18  20:35:00  20.0         CALLE   NaN   

    LUGAR_DEL_HECHO_NORMALIZADO  COMUNA longitud Latitud VICTIMA ACUSADO  
119                          SD       0        .       .  PEATON      SD  
           ID  N_VICTIMAS      FECHA HORA  HH TIPO_DE_CALLE           Calle  \
518  20190103           1 2019-12-18   SD NaN      GRAL PAZ  PAZ, GRAL. AV.   

    LUGAR_DEL_HECHO_NORMALIZADO  COMUNA      longitud       Latitud VICTIMA  \
518     PAZ, GRAL. AV. & GRIVEO      11  -58.52169422  -34.59471640    MOTO   

    ACUSADO  
518    MOTO  


In [14]:
homicidios.loc[homicidios['HH'].isnull(), 'HH'] = '00'
homicidios['HH'] = pd.to_numeric(homicidios['HH'])
homicidios.loc[homicidios['Calle'].isnull(), 'Calle'] = 'DESCONOCIDO'


C:\Users\OS\AppData\Local\Temp\ipykernel_15752\1592614858.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '00' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  homicidios.loc[homicidios['HH'].isnull(), 'HH'] = '00'


In [15]:
homicidios.loc[homicidios['HORA'] == 'SD', 'HORA'] = '00:00:00'
homicidios.loc[homicidios['VICTIMA'] == 'SD', 'VICTIMA'] = 'DESCONOCIDO'
homicidios.loc[homicidios['ACUSADO'] == 'SD', 'ACUSADO'] = 'DESCONOCIDO'

In [16]:
homicidios['Latitud'] = homicidios['Latitud'].str.replace(',', '.')
homicidios['longitud'] = homicidios['longitud'].str.replace(',', '.')
homicidios = homicidios[homicidios['Latitud'] != '.']
homicidios = homicidios[homicidios['longitud'] != '.']
homicidios['Latitud'] = homicidios['Latitud'].astype(float)
homicidios['longitud'] = homicidios['longitud'].astype(float)

In [17]:
homicidios['HH'] = homicidios['HH'].astype(int)

## Lesiones

este archivo requiere un trabajo mas extenso

tiene una cantidad de nulos y valores 'SD', y columnas que hay que tratar, eliminar o modificar para crear un archivo listo para analizar y cargar a SQL

In [19]:
lesiones['direc_normalizada'] = lesiones['otra_direccion'].apply(lambda x: x.lower() if pd.notnull(x) else x)


Las columnas:

**'aaaa','mm','dd'**

**'calle','altura','geocodificacion_CABA'**

**'participantes','moto','auto','transporte_publico'**

**'camion','ciclista','direccion_normalizada', 'cruce','otra_direccion'**

seran quitadas puesto no aportan nada relevante

In [20]:
columnas = ['aaaa','mm','dd','calle','altura','geocodificacion_CABA','participantes','moto','auto','transporte_publico','camion','ciclista','direccion_normalizada', 'cruce','otra_direccion']
lesiones = lesiones.drop(columns=columnas)

Se quitan los caracteres no numericos de **id**

se pasa a formato datetime a la columna **fecha**

se inserta la columna **direc_normalizada** antes de *longitud* y se le cambian los valores 'SD' a 'sin datos'

se toman los valores nulos, SD y 'No especificada' (hay uno solo de este ultimo) a 0, valor numerico que queda como indicador de desconocido

In [21]:
lesiones['id'] = lesiones['id'].str.replace(r'\D', '', regex=True)
lesiones['fecha'] = pd.to_datetime(lesiones['fecha']).dt.strftime('%Y-%m-%d')
columnas = lesiones.columns.tolist()
indice_longitud = columnas.index('longitud')
columnas.insert(indice_longitud, 'direc_normalizada')
lesiones = lesiones[columnas]
lesiones['direc_normalizada'] = lesiones['direc_normalizada'].replace('sd', 'sin datos', regex=True)
lesiones = lesiones.loc[:,~lesiones.columns.duplicated()]

In [22]:
lesiones['comuna'] = lesiones['comuna'].replace('SD', 0)
lesiones['comuna'] = lesiones['comuna'].replace('No Especificada', 0)
lesiones['comuna'] = lesiones['comuna'].fillna(0)
lesiones['comuna'] = pd.to_numeric(lesiones['comuna'])

C:\Users\OS\AppData\Local\Temp\ipykernel_15752\3351253537.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lesiones['comuna'] = lesiones['comuna'].replace('No Especificada', 0)


Le daremos formato de hora a la columna *hora*

la columna **tipo_Calle** no cuenta con valores nulos pero si muchos tipo 'SD' los cuales transformaremos en:

- AUTOPISTA
- AVENIDA
- CALLE
- GRAL PAZ

esto lo haremos en base a los valores de 'direc_normalizada'

In [24]:
def convertir_formato_hora(hora):
    hora_datetime = pd.to_datetime(hora, format='%H:%M:%S', errors='coerce')
    hora_24h = hora_datetime.strftime('%H:%M:%S') if not pd.isnull(hora_datetime) else None
    return hora_24h
lesiones['hora'] = lesiones['hora'].apply(convertir_formato_hora)

In [25]:
lesiones.loc[lesiones['direc_normalizada'].str.contains('autopista', case=False), 'tipo_calle'] = 'AUTOPISTA'
lesiones.loc[lesiones['direc_normalizada'].str.contains('av.', case=False), 'tipo_calle'] = 'AVENIDA'
lesiones.loc[lesiones['direc_normalizada'].str.contains('calle', case=False), 'tipo_calle'] = 'CALLE'
lesiones.loc[lesiones['direc_normalizada'] == 'paz, gral. av.', 'tipo_calle'] = 'GRAL PAZ'
lesiones.loc[lesiones['tipo_calle'] == 'SD', 'tipo_calle'] = 'CALLE'


para las columnas de **latitud** y **longitud** llenaremos los valores nulos con 0 y luego reemplazaramos los outliers y los valores iguales a 0 con la media de cada columna


In [ ]:
lesiones['latutid'] =lesiones['latutid'].fillna(00.00000)
lesiones['longitud'] =lesiones['longitud'].fillna(00.00000)
lesiones['latutid'] =lesiones['latutid'].replace('SD',00.00000)
lesiones['longitud'] =lesiones['longitud'].replace('SD',00.00000)

In [26]:
lesiones.rename(columns={'latutid': 'latitud'}, inplace=True)
lesiones['latitud'] = lesiones['latitud'].astype(float)
lesiones['longitud'] = lesiones['longitud'].astype(float)
lesiones['latitud'] = lesiones['latitud'].round(8)
lesiones['longitud'] = lesiones['longitud'].round(8)
lesiones.loc[lesiones['latitud'] == 0, 'latitud'] = lesiones['latitud'].replace(0, lesiones.loc[lesiones['latitud'] != 0, 'latitud'].mean()).round(8)
lesiones.loc[lesiones['longitud'] == 0, 'longitud'] = lesiones['longitud'].replace(0, lesiones.loc[lesiones['longitud'] != 0, 'longitud'].mean()).round(8)
pd.options.display.float_format = '{:.8f}'.format
lesiones.loc[23784, 'longitud'] = -58.29963273
lesiones.loc[23784, 'latitud'] = -34.72890716

In [27]:
lesiones['comuna'] = lesiones['comuna'].astype(int)
lesiones['n_victimas'] = lesiones['n_victimas'].astype(int)
lesiones.loc[lesiones['gravedad'] == 'SD', 'gravedad'] = 'Sin Gravedad'
lesiones.loc[lesiones['acusado'] == 'SD', 'acusado'] = 'desconocido'
lesiones.loc[lesiones['victima'] == 'SD', 'victima'] = 'desconocido'

In [28]:
# errores al cargar long y lat
long_med_f = lesiones.loc[(lesiones['longitud'] > -180.0) & (lesiones['longitud'] < 180.0), 'longitud']
lat_med_f = lesiones.loc[(lesiones['latitud'] > -180.0) & (lesiones['latitud'] < 180.0), 'latitud']
long_med = round(long_med_f.mean(),8)
lat_med = round(lat_med_f.mean(),8)
lesiones.loc[lesiones['longitud'] < -180.0, 'longitud'] = long_med
lesiones.loc[lesiones['longitud'] > 180.0, 'longitud'] = long_med
lesiones.loc[lesiones['latitud'] < -180.0, 'latitud'] = lat_med
lesiones.loc[lesiones['latitud'] > 180.0, 'latitud'] = lat_med

In [29]:
# manejo de nulos en hora y despues se llenara franja_hora
lesiones['franja_hora'] = lesiones['franja_hora'].replace(['sd','SD'], pd.NA)
med = round(lesiones['franja_hora'].mean(skipna= True))
lesiones['franja_hora'].fillna(med, inplace= True)
lesiones['hora'] = lesiones['hora'].fillna(lesiones['franja_hora'].astype(str).str[:2] + ':00:00')

C:\Users\OS\AppData\Local\Temp\ipykernel_15752\3302146950.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  lesiones['franja_hora'].fillna(med, inplace= True)
C:\Users\OS\AppData\Local\Temp\ipykernel_15752\3302146950.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lesiones['franja_hora'].fillna(med, inpla

In [31]:
homicidios.to_csv("homicidios.csv",index=False)
lesiones.to_csv("lesiones.csv", index=False)